### 1.1 Importar Libs

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os
import json
import pickle
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, confusion_matrix, 
                           accuracy_score, precision_score, recall_score, 
                           f1_score, roc_auc_score, roc_curve)
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform, randint
from joblib import dump, load
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Importar funções dos módulos customizados
from ml_utils import gmean_score, evaluate_model, load_and_prepare_datasets
from search_utils import (plot_search_history, multiple_randomized_search,
                          plot_search_history_from_loaded, 
                          load_search_results, get_best_params_from_saved,
                          save_search_results, save_final_results)

# Configurações de plotagem
plt.rcParams['figure.figsize'] = [12, 8]
sns.set_style("whitegrid")

print("Bibliotecas importadas com sucesso!")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Scikit-learn: {sklearn.__version__}")
print(f"XGBoost: {xgb.__version__}")

Bibliotecas importadas com sucesso!
Pandas: 2.3.2
NumPy: 2.3.3
Scikit-learn: 1.7.2
XGBoost: 3.1.2


### 1.2 Definir Nome do Modelo

In [ ]:
# Definir nome do modelo para uso em salvamento e exibição
MODEL_NAME = "Stacking"
print(f"Modelo: {MODEL_NAME}")

### 1.3 Definir Comitê Heterogêneo (Stacking)

In [20]:
# ======================================================================
# COMITÊ HETEROGÊNEO (STACKING) - IMPLEMENTAÇÃO
# ======================================================================

from sklearn.base import BaseEstimator, ClassifierMixin

class HeterogeneousStackingCommittee(BaseEstimator, ClassifierMixin):
    def __init__(self, 
                 # Parâmetros Decision Tree (variáveis)
                 dt_max_depth=None, dt_min_samples_split=2, dt_min_samples_leaf=1,
                 # Parâmetros Decision Tree (fixos)
                 dt_criterion='gini', dt_max_features='sqrt',
                 # Parâmetros Random Forest (variáveis)
                 rf_n_estimators=100, rf_max_depth=None,
                 # Parâmetros Random Forest (fixos)
                 rf_criterion='gini', rf_max_features='sqrt',
                 rf_min_samples_split=16, rf_min_samples_leaf=1,
                 # Parâmetros XGBoost (variáveis)
                 xgb_n_estimators=100, xgb_max_depth=6, xgb_learning_rate=0.3,
                 # Parâmetros XGBoost (fixos - melhores valores)
                 xgb_subsample=0.6527464393047274, xgb_colsample_bytree=0.6502374440504688,
                 xgb_min_child_weight=1, xgb_gamma=0.13697900853304845,
                 xgb_reg_alpha=0.778102210605468, xgb_reg_lambda=1.518712387770365,
                 # Parâmetros do meta-estimador
                 meta_C=1.0, meta_max_iter=1000,
                 # Configurações gerais
                 cv=5, random_state=None):
        """
        Comitê Heterogêneo usando StackingClassifier com árvores de decisão
        
        Estimadores base: Decision Tree, Random Forest, XGBoost
        Meta-estimador: Logistic Regression
        """
        # Parâmetros Decision Tree
        self.dt_max_depth = dt_max_depth
        self.dt_min_samples_split = dt_min_samples_split
        self.dt_min_samples_leaf = dt_min_samples_leaf
        self.dt_criterion = dt_criterion
        self.dt_max_features = dt_max_features
        
        self.rf_n_estimators = rf_n_estimators
        self.rf_max_depth = rf_max_depth
        self.rf_criterion = rf_criterion
        self.rf_max_features = rf_max_features
        self.rf_min_samples_split = rf_min_samples_split
        self.rf_min_samples_leaf = rf_min_samples_leaf
        
        # Parâmetros XGBoost
        self.xgb_n_estimators = xgb_n_estimators
        self.xgb_max_depth = xgb_max_depth
        self.xgb_learning_rate = xgb_learning_rate
        self.xgb_subsample = xgb_subsample
        self.xgb_colsample_bytree = xgb_colsample_bytree
        self.xgb_min_child_weight = xgb_min_child_weight
        self.xgb_gamma = xgb_gamma
        self.xgb_reg_alpha = xgb_reg_alpha
        self.xgb_reg_lambda = xgb_reg_lambda
        
        # Parâmetros do meta-estimador
        self.meta_C = meta_C
        self.meta_max_iter = meta_max_iter
        
        # Configurações gerais
        self.cv = cv
        self.random_state = random_state
        
    def fit(self, X, y):
        # Definir estimadores base com parâmetros otimizáveis
        base_estimators = [
            ('decision_tree', DecisionTreeClassifier(
                max_depth=self.dt_max_depth,
                min_samples_split=self.dt_min_samples_split,
                min_samples_leaf=self.dt_min_samples_leaf,
                criterion=self.dt_criterion,
                max_features=self.dt_max_features,
                random_state=self.random_state
            )),
            ('random_forest', RandomForestClassifier(
                n_estimators=self.rf_n_estimators,
                max_depth=self.rf_max_depth,
                criterion=self.rf_criterion,
                max_features=self.rf_max_features,
                min_samples_split=self.rf_min_samples_split,
                min_samples_leaf=self.rf_min_samples_leaf,
                random_state=self.random_state
            )),
            ('xgboost', xgb.XGBClassifier(
                n_estimators=self.xgb_n_estimators,
                max_depth=self.xgb_max_depth,
                learning_rate=self.xgb_learning_rate,
                subsample=self.xgb_subsample,
                colsample_bytree=self.xgb_colsample_bytree,
                min_child_weight=self.xgb_min_child_weight,
                gamma=self.xgb_gamma,
                reg_alpha=self.xgb_reg_alpha,
                reg_lambda=self.xgb_reg_lambda,
                objective='binary:logistic',
                eval_metric='logloss',
                random_state=self.random_state,
                verbosity=0
            ))
        ]
        
        # Meta-estimador (Logistic Regression)
        meta_estimator = LogisticRegression(
            C=self.meta_C,
            max_iter=self.meta_max_iter,
            random_state=self.random_state
        )
        
        # Criar o StackingClassifier
        self.stacking_classifier = StackingClassifier(
            estimators=base_estimators,
            final_estimator=meta_estimator,
            cv=self.cv,
            stack_method='predict_proba',  # Usar probabilidades
            n_jobs=1  # Evitar conflitos de paralelização
        )
        
        # Treinar o ensemble
        self.stacking_classifier.fit(X, y)
        self.classes_ = self.stacking_classifier.classes_
        
        return self
    
    def predict(self, X):
        return self.stacking_classifier.predict(X)
    
    def predict_proba(self, X):
        return self.stacking_classifier.predict_proba(X)
    
    def score(self, X, y):
        return accuracy_score(y, self.predict(X))

print("Comitê Heterogêneo (Stacking) com Decision Tree, Random Forest e XGBoost definido com sucesso!")

Comitê Heterogêneo (Stacking) com Decision Tree, Random Forest e XGBoost definido com sucesso!


### 1.4 Carregar Datasets

In [ ]:
# Carregamento e preparação inicial dos dados
print("Carregando datasets...")

# Carregar e preparar datasets usando função do módulo
(X_train, X_test, y_train, y_test, 
 X_train_scaled, X_test_scaled, 
 train_data, test_data, scaler) = load_and_prepare_datasets()

print(f"Dataset de treino: {train_data.shape}")
print(f"Dataset de teste: {test_data.shape}")

print("\nDistribuição das classes:")
print("Treino:", y_train.value_counts().to_dict())
print("Teste:", y_test.value_counts().to_dict())

Carregando datasets...


Dataset de treino: (853006, 19)
Dataset de teste: (215171, 19)

Distribuição das classes:
Treino: {0.0: 831112, 1.0: 21894}
Teste: {0.0: 209675, 1.0: 5496}

Distribuição das classes:
Treino: {0.0: 831112, 1.0: 21894}
Teste: {0.0: 209675, 1.0: 5496}


In [10]:
train_data.head()

,Hour,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BUN,WBC,Platelets,Gender,Unit1,Unit2,HospAdmTime,ICULOS,Critical_Risk_Window,Time_Category,SepsisLabel
0,8,-1.109794,-0.460180,-0.936182,2.873365,3.044201,2.159974,-0.073601,-0.277186,-1.393580,0.450716,1.0,1.0,0.0,-12.06,9.0,0,0,0.0
1,47,0.569971,-2.437770,0.173477,0.393960,0.650783,0.430943,-0.997324,0.309171,0.245616,-0.275108,1.0,1.0,0.0,-0.05,48.0,0,1,0.0
2,6,0.150030,0.978068,0.016114,-0.983487,-0.553850,-0.198683,-0.073601,-0.310739,0.121368,-0.191266,1.0,1.0,0.0,-0.02,7.0,0,0,0.0
3,39,-0.269912,0.258944,0.289355,0.853109,0.405094,-0.115460,0.752015,0.298591,-0.003965,-3.152920,0.0,0.0,1.0,-75.85,43.0,0,1,0.0
4,127,0.569971,-0.460180,0.007012,0.761279,1.834255,1.055050,-0.520987,0.702140,-0.180545,0.030433,0.0,0.0,1.0,-0.03,128.0,1,2,0.0


## 2. Sampling para Busca de Hiperparâmetros

In [24]:
# ======================================================================
# SAMPLING ESTRATIFICADO PARA BUSCA DE HIPERPARÂMETROS
# ======================================================================

print("=== PREPARAÇÃO DE AMOSTRA PARA BUSCA DE HIPERPARÂMETROS ===")

# Amostra estratificada do dataset de treino (muito pequena devido à complexidade)
_, X_sample, _, y_sample = train_test_split(
    X_train_scaled, y_train, 
    test_size=0.01, 
    stratify=y_train,
    random_state=10
)

print(f"Dataset original de treino: {X_train_scaled.shape[0]:,} amostras")
print(f"Amostra para busca de hiperparâmetros: {X_sample.shape[0]:,} amostras")
print(f"Redução: {(1 - X_sample.shape[0]/X_train_scaled.shape[0])*100:.1f}%")

print("\nDistribuição das classes na amostra:")
print("Amostra:", pd.Series(y_sample).value_counts().to_dict())
print("Original:", y_train.value_counts().to_dict())

=== PREPARAÇÃO DE AMOSTRA PARA BUSCA DE HIPERPARÂMETROS ===
Dataset original de treino: 853,006 amostras
Amostra para busca de hiperparâmetros: 8,531 amostras
Redução: 99.0%

Distribuição das classes na amostra:
Amostra: {0.0: 8312, 1.0: 219}
Original: {0.0: 831112, 1.0: 21894}
Dataset original de treino: 853,006 amostras
Amostra para busca de hiperparâmetros: 8,531 amostras
Redução: 99.0%

Distribuição das classes na amostra:
Amostra: {0.0: 8312, 1.0: 219}
Original: {0.0: 831112, 1.0: 21894}


## 3.1 Definir Folds

In [ ]:
# Configuração da validação cruzada estratificada
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## 4. Stacking - Busca de Hiperparâmetros

In [25]:
# ======================================================================
# 4.1 BUSCA DE HIPERPARAMETROS 
# ======================================================================

print(f"=== BUSCA DE HIPERPARÂMETROS - {MODEL_NAME} ===")

# Definição do Espaço de Hiperparâmetros para Stacking
param_distributions = {
    # Parâmetros Decision Tree (busca)
    'dt_max_depth': randint(3, 50),
    'dt_min_samples_split': randint(2, 40),
    'dt_min_samples_leaf': randint(1, 50),
    # Parâmetros Decision Tree fixos: criterion='gini', max_features='sqrt'
    
    # Parâmetros Random Forest (busca)
    'rf_n_estimators': randint(5, 300),
    'rf_max_depth': randint(3, 50),
    # Parâmetros Random Forest fixos: criterion='gini', max_features='sqrt', min_samples_split=16, min_samples_leaf=1
    
    # Parâmetros XGBoost (busca apenas n_estimators, max_depth, learning_rate)
    # Demais parâmetros fixos nos melhores valores encontrados
    'xgb_n_estimators': randint(140, 381),
    'xgb_max_depth': randint(3, 11),
    'xgb_learning_rate': uniform(0.5, 1.5),
    
    # Parâmetros do Meta-estimador (Logistic Regression)
    'meta_C': uniform(0.001, 10),  # Regularização: valores menores = mais regularização
    'meta_max_iter': [100, 500, 1000, 2000, 3000],  # Iterações máximas para convergência
}

# Múltiplas execuções do RandomizedSearchCV
print(f"Iniciando busca de hiperparâmetros para {MODEL_NAME}...")
model_search, model_all_searches, best_params = multiple_randomized_search(
    estimator=HeterogeneousStackingCommittee(random_state=42, cv=3),
    param_distributions=param_distributions,
    X=X_sample,                  
    y=y_sample,
    cv_strategy=cv_strategy,
    n_searches=20,  
    n_iter_per_search=2,       
    scoring='f1',
    n_jobs=1,  # Processamento sequencial
)

# Seleção da Melhor Configuração
print(f"\n--- RESULTADOS {MODEL_NAME} ---")
print("Melhores hiperparâmetros:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

print(f"\nMelhor F1-Score (CV): {model_search.best_score_:.4f}")

=== BUSCA DE HIPERPARÂMETROS - Stacking ===
Iniciando busca de hiperparâmetros para Stacking...
Executando 20 buscas com 2 iterações cada...

Busca 1/20...
Melhor score desta busca: 0.0265
Melhor configuração desta busca: {'dt_max_depth': 4, 'dt_min_samples_leaf': 18, 'dt_min_samples_split': 34, 'meta_C': np.float64(3.176949833122895), 'meta_max_iter': 1000, 'rf_max_depth': 4, 'rf_n_estimators': 233, 'xgb_learning_rate': np.float64(1.7410978906984163), 'xgb_max_depth': 8, 'xgb_n_estimators': 269}
Melhor score geral até agora: 0.0265

Busca 2/20...
Melhor score desta busca: 0.0265
Melhor configuração desta busca: {'dt_max_depth': 4, 'dt_min_samples_leaf': 18, 'dt_min_samples_split': 34, 'meta_C': np.float64(3.176949833122895), 'meta_max_iter': 1000, 'rf_max_depth': 4, 'rf_n_estimators': 233, 'xgb_learning_rate': np.float64(1.7410978906984163), 'xgb_max_depth': 8, 'xgb_n_estimators': 269}
Melhor score geral até agora: 0.0265

Busca 2/20...
Melhor score desta busca: 0.0263
Melhor configur

KeyboardInterrupt: 

In [ ]:
# Registro de Desempenho - plotar evolução
plot_search_history(model_all_searches, model_search, MODEL_NAME)

In [ ]:
# ======================================================================
# 4.5 ANÁLISE DAS MELHORES CONFIGURAÇÕES ENCONTRADAS
# ======================================================================

print(f"=== TOP CONFIGURAÇÕES - {MODEL_NAME} ===")

# Extrair os melhores resultados de cada busca
best_configs = []

for i, search_result in enumerate(model_all_searches):
    config = {
        'Busca': i + 1,
        'F1_Score': search_result['best_score'],
        'RF_N_Est': search_result['best_params']['rf_n_estimators'],
        'RF_Depth': search_result['best_params']['rf_max_depth'],
        'SVM_C': search_result['best_params']['svm_C'],
        'SVM_Gamma': search_result['best_params']['svm_gamma'],
        'KNN_K': search_result['best_params']['knn_n_neighbors'],
        'MLP_Layers': str(search_result['best_params']['mlp_hidden_layers']),
        'Meta_C': search_result['best_params']['meta_C'],
        'CV_Folds': search_result['best_params']['cv']
    }
    best_configs.append(config)

# Converter para DataFrame e ordenar por F1-Score
results_df = pd.DataFrame(best_configs)
results_df = results_df.sort_values('F1_Score', ascending=False).reset_index(drop=True)
results_df['Ranking'] = range(1, len(results_df) + 1)

# Reordenar colunas
results_df = results_df[['Ranking', 'Busca', 'F1_Score', 'RF_N_Est', 'RF_Depth', 
                        'SVM_C', 'SVM_Gamma', 'KNN_K', 'MLP_Layers', 'Meta_C', 'CV_Folds']]

# Mostrar tabela formatada
print("Configurações encontradas (ordenadas por F1-Score Binário):")
print("-" * 150)
print(results_df.to_string(index=False, float_format='%.4f'))

# Estatísticas resumidas
print(f"\n--- ESTATÍSTICAS DAS CONFIGURAÇÕES ---")
print(f"Melhor F1-Score: {results_df['F1_Score'].max():.4f}")
print(f"F1-Score médio: {results_df['F1_Score'].mean():.4f}")
print(f"Desvio padrão: {results_df['F1_Score'].std():.4f}")
print(f"F1-Score mínimo: {results_df['F1_Score'].min():.4f}")

## 5. Salvar Resultados de Busca

In [ ]:
# Salvar Resultados da Busca de Hiperparâmetros usando função do módulo
search_df = save_search_results(
    model_name=MODEL_NAME,
    model_search=model_search,
    model_all_searches=model_all_searches,
    n_searches=5,
    n_iter_per_search=3,
    scoring='f1',
    cv_folds=3,
    top_params_columns=['rf_n_estimators', 'svm_C', 'knn_n_neighbors', 'meta_C', 'cv'],
    searches_folder='searches'
)

## 5.2 Carregar Resultado de busca

In [ ]:
#### 4.2 Carregar Resultados Salvos (Função Auxiliar)
# Exemplo de uso da função (não executar se já temos os resultados)
loaded_results = load_search_results(MODEL_NAME)

In [ ]:
# Plotar a história da busca a partir dos resultados carregados
plot_search_history_from_loaded(loaded_results, MODEL_NAME)

In [ ]:

#### 4.3 Recuperar Melhores Parâmetros para Uso Posterior
# Exemplo de uso (descomente se precisar carregar parâmetros salvos):
if 'loaded_results' in locals():
    best_params = get_best_params_from_saved(MODEL_NAME)
    if best_params:
        print(f"✅ Parâmetros carregados: {best_params}")
    best_score = loaded_results['summary']['best_overall_score']
    print(f"✅ Melhor F1-Score carregado: {best_score:.4f}")
else:
    best_params = model_search.best_params_
    best_score = model_search.best_score_
    print(f"✅ Usando parâmetros da busca atual: {best_params}")
    print(f"✅ Melhor F1-Score da busca atual: {best_score:.4f}")

## 6. Treinar Modelo Final e Salvar

In [ ]:
# Treinamento Final com melhores hiperparâmetros
best_model = HeterogeneousStackingCommittee(**best_params, random_state=42)
best_model.fit(X_train_scaled, y_train)

print(f"\nModelo final {MODEL_NAME} treinado: {best_model}")

# Criar pasta se não existir
os.makedirs('models', exist_ok=True)

# Salvar modelo treinado
dump(best_model, f'models/{MODEL_NAME.lower()}_trained.joblib')
print(f"Modelo salvo: models/{MODEL_NAME.lower()}_trained.joblib")

## 7. Avaliação Final e Salvamento dos Resultados

In [ ]:
# Carregar modelo
loaded_model = load(f'models/{MODEL_NAME.lower()}_trained.joblib')

In [ ]:
print(f"=== AVALIAÇÃO E SALVAMENTO DOS RESULTADOS - {MODEL_NAME} ===")

# Criar pastas se não existirem
os.makedirs('results', exist_ok=True)

# Avaliação completa do modelo
print("\nAvaliando performance do modelo...")

if 'loaded_model' in locals():
    model = loaded_model
else:
    model = best_model

X_train_eval = X_train_scaled
y_train_eval = y_train
X_test_eval = X_test_scaled
y_test_eval = y_test

# Avaliar modelo
train_metrics, test_metrics, y_pred = evaluate_model(
    model, X_train_eval, X_test_eval, y_train_eval, y_test_eval, MODEL_NAME
)

In [ ]:
# Salvar resultados finais usando função do módulo
model_final_results = save_final_results(
    model_name=MODEL_NAME,
    best_params=best_params,
    best_score=best_score,
    train_metrics=train_metrics,
    test_metrics=test_metrics,
    y_pred=y_pred,
    y_test=y_test_eval,
    X_train_scaled=X_train_eval,
    X_test_scaled=X_test_eval,
    results_folder='results'
)

# Mostrar resumo
print(f"\n--- RESUMO {MODEL_NAME} ---")
print(f"F1-Score CV: {model_final_results['best_cv_score']:.4f}")
print(f"F1-Score Teste: {test_metrics['f1']:.4f}")
print(f"Acurácia Teste: {test_metrics['accuracy']:.4f}")
print(f"Precisão Teste: {test_metrics['precision']:.4f}")
print(f"Recall Teste: {test_metrics['recall']:.4f}")
print(f"G-Mean Teste: {test_metrics['gmean']:.4f}")
if test_metrics['auc_roc']:
    print(f"AUC-ROC Teste: {test_metrics['auc_roc']:.4f}")